### 라이브러리 선언

In [1]:
# 숫자 및 데이터 다루는 라이브러리
import numpy as np
import pandas as pd

# 분류모델 선언
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier

# 훈련/테스트 데이터 자동 분리
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer


# 분류모델 정확도 라이브러리 선언
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

# 데이터 시각화
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.tree import export_text
from sklearn.tree import plot_tree

### 데이터 불러오기

In [2]:
### CSV 파일을 읽어 DataFrame 변수에 저장하기
### 1페이지_이름 순 정렬
ageData1 = pd.read_csv("../we/gundae1_score_final.csv", encoding="ms949")

### 2페이지_이름 순 정렬
ageData2 = pd.read_csv("../we/gundae2_score_final.csv", encoding="ms949")

### 3페이지_이름 순 정렬
ageData3 = pd.read_csv("../we/gundae3_score_final.csv", encoding="ms949")

### 4페이지_이름 순 정렬
ageData4 = pd.read_csv("../we/gundae4_score_final.csv", encoding="ms949")

### 5페이지_이름 순 정렬
ageData5 = pd.read_csv("../we/gundae5_score_final.csv", encoding="ms949")

### 6페이지_이름 순 정렬
ageData6 = pd.read_csv("../we/gundae6_score_final6.csv", encoding="ms949")

### 1. 데이터 병합

In [3]:
merge12 = pd.merge(ageData1, ageData2, how="outer", on=None)
merge34 = pd.merge(ageData3, ageData4, how="outer", on=None)
merge56 = pd.merge(ageData5, ageData6, how="outer", on=None)
merge1234 = pd.merge(merge12,merge34, how="outer", on=None)

mergeData1 = pd.merge(merge1234,merge56, how="outer", on=None)

In [4]:
# 중복 제거
mergeData1.drop_duplicates()

,Unnamed: 0,name,gubun,score,visitCount,blogCount
0,0,카페 아르무아,"카페,디저트",4.44/5,방문자 리뷰_ 672,블로그 리뷰_ 989
1,1,고향양꼬치,양꼬치,4.38/5,방문자 리뷰_ 226,"블로그 리뷰_ 1,096"
2,2,고베규카츠 건대점,돈가스,3.64/5,방문자 리뷰_ 182,블로그 리뷰_ 64
3,3,송화산시도삭면,"딤섬,중식만두",4.53/5,"방문자 리뷰_ 1,050","블로그 리뷰_ 1,427"
4,4,서북면옥,냉면,4.26/5,"방문자 리뷰_ 1,200","블로그 리뷰_ 1,401"
...,...,...,...,...,...,...
300,45,선명,요리주점,4.88/5,방문자 리뷰_ 40,블로그 리뷰_ 15
301,46,에스프레소 바 시 자양점,카페,4.41/5,방문자 리뷰_ 374,블로그 리뷰_ 33
302,47,굽네치킨 구의1동점,"치킨,닭강정",4.38/5주문자리뷰 147블로그리뷰 15,"방문자 리뷰_굽네치킨 구의1동점치킨,닭강정\n별점\n4.38/5주문자리뷰 147",블로그 리뷰_ 15
303,48,푸른술집할라 건대점,요리주점,4.38/5주문자리뷰 147블로그리뷰 15,방문자 리뷰_ 37,블로그 리뷰_ 31


In [5]:
mergeData1.shape

(305, 6)

In [6]:
mergeData1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 305 entries, 0 to 304
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  305 non-null    int64 
 1   name        305 non-null    object
 2   gubun       305 non-null    object
 3   score       305 non-null    object
 4   visitCount  305 non-null    object
 5   blogCount   305 non-null    object
dtypes: int64(1), object(5)
memory usage: 16.7+ KB


In [7]:
### 데이터 전처리
mergeData1["score"] = mergeData1.score.str.split('/').str[0] # 별점 점수
mergeData1["visitCount"] = mergeData1.visitCount.str.split('_').str[1] # 방문자 리뷰 수
mergeData1["blogCount"] = mergeData1.blogCount.str.split('_').str[1] # 블로그 리뷰 수

In [8]:
### 리뷰 수 콤마 제거 (int형으로 변환하기위함)
mergeData1["visitCount"] = mergeData1.visitCount.str.replace(',', '')
mergeData1["blogCount"] = mergeData1.blogCount.str.replace(',', '')

In [9]:
mergeData1

,Unnamed: 0,name,gubun,score,visitCount,blogCount
0,0,카페 아르무아,"카페,디저트",4.44,672,989
1,1,고향양꼬치,양꼬치,4.38,226,1096
2,2,고베규카츠 건대점,돈가스,3.64,182,64
3,3,송화산시도삭면,"딤섬,중식만두",4.53,1050,1427
4,4,서북면옥,냉면,4.26,1200,1401
...,...,...,...,...,...,...
300,45,선명,요리주점,4.88,40,15
301,46,에스프레소 바 시 자양점,카페,4.41,374,33
302,47,굽네치킨 구의1동점,"치킨,닭강정",4.38,굽네치킨 구의1동점치킨닭강정\n별점\n4.38/5주문자리뷰 147,15
303,48,푸른술집할라 건대점,요리주점,4.38,37,31


In [10]:
mergeData1.visitCount.loc[mergeData1.visitCount.str.contains("주문자리뷰")] = 147

C:\Users\SMART18\Anaconda3\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [11]:
mergeData1

,Unnamed: 0,name,gubun,score,visitCount,blogCount
0,0,카페 아르무아,"카페,디저트",4.44,672,989
1,1,고향양꼬치,양꼬치,4.38,226,1096
2,2,고베규카츠 건대점,돈가스,3.64,182,64
3,3,송화산시도삭면,"딤섬,중식만두",4.53,1050,1427
4,4,서북면옥,냉면,4.26,1200,1401
...,...,...,...,...,...,...
300,45,선명,요리주점,4.88,40,15
301,46,에스프레소 바 시 자양점,카페,4.41,374,33
302,47,굽네치킨 구의1동점,"치킨,닭강정",4.38,147,15
303,48,푸른술집할라 건대점,요리주점,4.38,37,31


In [12]:
### 타입 통합
mergeData1 =  mergeData1.astype( { "score" : float, "visitCount" : int, "blogCount" : int })

In [13]:
mergeData1

,Unnamed: 0,name,gubun,score,visitCount,blogCount
0,0,카페 아르무아,"카페,디저트",4.44,672,989
1,1,고향양꼬치,양꼬치,4.38,226,1096
2,2,고베규카츠 건대점,돈가스,3.64,182,64
3,3,송화산시도삭면,"딤섬,중식만두",4.53,1050,1427
4,4,서북면옥,냉면,4.26,1200,1401
...,...,...,...,...,...,...
300,45,선명,요리주점,4.88,40,15
301,46,에스프레소 바 시 자양점,카페,4.41,374,33
302,47,굽네치킨 구의1동점,"치킨,닭강정",4.38,147,15
303,48,푸른술집할라 건대점,요리주점,4.38,37,31


In [16]:
mergeData1 = mergeData1.sort_values(by=["name"], ignore_index=True)

In [19]:
mergeData1 = mergeData1.drop(columns=["Unnamed: 0"])

In [20]:
mergeData1

,name,gubun,score,visitCount,blogCount
0,60년전통신촌황소곱창 건대직영점,"곱창,막창,양",4.12,297,385
1,972Bar,바(BAR),4.58,95,438
2,C156,양식,4.59,729,514
3,CUCINA843,양식,4.59,74,150
4,LITE LANE,카페,4.43,458,260
...,...,...,...,...,...
300,후토,이자카야,4.47,147,148
301,훈춘양꼬치,양꼬치,4.64,133,123
302,흐릇,카페,4.75,74,44
303,흐릇,카페,4.75,74,44


In [23]:
mergeData1.to_csv("fixed_건대_Category.csv",mode="w", encoding = "ms949")